In [1]:
using LinearAlgebra

In [9]:
ρ = [1 2 3; 4 5 6 ; 7 8 9]

3×3 Matrix{Int64}:
 1  2  3
 4  5  6
 7  8  9

In [29]:
size(ρ)

(3, 3)

In [10]:
r = vec(ρ)

9-element Vector{Int64}:
 1
 4
 7
 2
 5
 8
 3
 6
 9

In [11]:
reshape(r, (3,3))

3×3 Matrix{Int64}:
 1  2  3
 4  5  6
 7  8  9

In [24]:
function vectorize(x)
    return vec(x)
end

function unvectorize(x)
    # Assumes the input object to be a reshapable to a square matrix.
    d = trunc(Int, sqrt(length(x)))
    return reshape(x, (d,d))
end


unvectorize (generic function with 1 method)

In [25]:
ρ = [1 2 3; 4 5 6 ; 7 8 9]

3×3 Matrix{Int64}:
 1  2  3
 4  5  6
 7  8  9

In [26]:
vectorize(ρ)

9-element Vector{Int64}:
 1
 4
 7
 2
 5
 8
 3
 6
 9

In [27]:
unvectorize(vectorize(ρ))

3×3 Matrix{Int64}:
 1  2  3
 4  5  6
 7  8  9

In [30]:
function conjugate_matrix(x)
    return conj.(x)
end

conjugate_matrix (generic function with 1 method)

In [42]:
function lindblad_operator_to_vectorized_form(L)
    d = size(L)[1]
    ide = Matrix{Float64}(I, d, d)
    return kron(conjugate_matrix(L), L) - 0.5 * kron(transpose(L' * L), ide) - 0.5 * kron(ide, L' * L)
end

lindblad_operator_to_vectorized_form (generic function with 1 method)

In [44]:
function unitary_evolution_to_vectorized_form(H)
    d = size(H)[1]
    ide = Matrix{Float64}(I, d, d)
    return - 1im * (kron(ide, H) -  kron(transpose(H), ide))
end

unitary_evolution_to_vectorized_form (generic function with 1 method)

In [45]:
function liouvillian(H, C_l)
    L = unitary_evolution_to_vectorized_form(H)
    for C in C_l
        L += lindblad_operator_to_vectorized_form(C)
    end
    return L
end

liouvillian (generic function with 1 method)

In [46]:
function adjoint_liouvillian(H, C_l)
    d = size(H)[1]
    ide = Matrix{Float64}(I, d, d)

    hamiltonian_part = 1im * kron(ide, H) - 1im * kron(transpose(H), ide)
    L_dagger = hamiltonian_part

    for C in C_l
        L_dagger += kron(transpose(L), L') - 0.5 * kron(transpose(L' * L), ide) - 0.5 * kron(ide, L' * L)
    end
end

adjoint_liouvillian (generic function with 1 method)